In [ ]:
!pip install googletrans

In [156]:
!pip install openpyxl

    100% |████████████████████████████████| 163kB 1.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for openpyxl ... done
  Stored in directory: /Users/hsw/Library/Caches/pip/wheels/8d/ae/96/0b8e7890053c46c22b48d021d104b00e5544c3aedd641749e1
  Running setup.py bdist_wheel for jdcal ... done
  Stored in directory: /Users/hsw/Library/Caches/pip/wheels/0f/63/92/19ac65ed64189de4d662f269d39dd08a887258842ad2f29549
  Running setup.py bdist_wheel for et-xmlfile ... done
  Stored in directory: /Users/hsw/Library/Caches/pip/wheels/99/f6/53/5e18f3ff4ce36c990fa90ebdf2b80cd9b44dc461f750a1a77c
Successfully built openpyxl jdcal et-xmlfile


In [157]:
import openpyxl

In [ ]:
!pip install textblob

In [64]:
from googletrans import Translator
translator = Translator()

from newspaper import Article

from __future__ import print_function
from lexrankr import LexRank

##########################################

from selenium import webdriver

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from urllib import parse

import time
import random


import pandas as pd
import numpy as np

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Komoran

from operator import itemgetter

import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [210]:
driver = webdriver.Chrome('/Users/hsw/Desktop/개인파일/Text_Mining/Podbbang_Crawling/chromedriver')  ## 크롬을 열 수 있는 driver
driver.implicitly_wait(3)  ## 잠시 기다린 후
driver.get('https://azure.microsoft.com/ko-kr/services/cognitive-services/text-analytics/') ## 홈페이지 열기(실제 열리는 걸 확인할 수 있습니다.)

In [114]:
driver2 = webdriver.Chrome('/Users/hsw/Desktop/개인파일/Text_Mining/Podbbang_Crawling/chromedriver')  ## 크롬을 열 수 있는 driver
driver2.implicitly_wait(3)  ## 잠시 기다린 후
driver2.get('https://search.naver.com/search.naver?where=news&query=%ED%95%B8%EB%94%94%EC%86%8C%ED%94%84%ED%8A%B8&ie=utf8&sm=tab_opt&sort=1&photo=0&field=1&reporter_article=&pd=3&ds=2016.01.01&de=2017.11.15&docid=&nso=so%3Add%2Cp%3Afrom20140101to20171115%2Ca%3At&mynews=1&mson=0&refresh_start=0&related=0') ## 홈페이지 열기(실제 열리는 걸 확인할 수 있습니다.)

In [ ]:
def Find_URL(Company_name, pageNum): ##1번째 Function, 회사이름과 원하는 Page 입력하면 Title에 회사이름이 들어간 URL 긁어옴
    allResult = []
    transform = parse.quote(Company_name) 
    for num in range(1, (pageNum+1)):
        news = 'http://search.khan.co.kr/search.html?stb=khan&q=' + transform + '&pg=' + str(num) + '&sort=1'
        req = Request(news)
        response = urlopen(req)
        thepage = response.read()
        bsObj = BeautifulSoup(thepage,'html.parser')
        article = bsObj.findAll('dl', {'class':'phArtc'})
        for i in article:
            if(i.find('a').text.find(Company_name) == -1):  ## Title에 회사이름이 들어가있지 않으면
                continue ## Pass
            allResult.append([i.find('a').text, i.find('span').text[1:][:-1], 
                              i.find('a').attrs['href']])
    return(allResult)

In [73]:
def Find_URL2(Company_name, end):
    result = []
    end2 = end.split('.')
    end3 = datetime(int(end2[0]), int(end2[1]), int(end2[2]))
    begin = (end3 - relativedelta(months = 12)).strftime("%Y/%m/%d")
    end4 = (end3 - relativedelta(months = 3)).strftime("%Y/%m/%d")
    beginDate = "".join(begin.split('/'))
    endDate = "".join(end4.split('/'))
    
    
    driver2.find_element_by_xpath('//*[@id="news_inner_search_query"]').send_keys(Company_name)
    driver2.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[3]/span/button').click()
    time.sleep(1)
    driver2.find_element_by_xpath('//*[@id="snb"]/div/ul/li[2]/a').click()
    time.sleep(1)
    
    for i in beginDate:
        driver2.find_element_by_xpath('//*[@id="news_input_period_begin"]').send_keys(i)
        time.sleep(0.2)
    time.sleep(1)
    driver2.find_element_by_xpath('//*[@id="news_input_period_end"]').click()
    time.sleep(1)
    for j in endDate:
        driver2.find_element_by_xpath('//*[@id="news_input_period_end"]').send_keys(j)
        time.sleep(0.2)
    time.sleep(1)
    driver2.find_element_by_xpath('//*[@id="_nx_option_date"]/div[2]/span/button/span').click()
    time.sleep(1)
    
    
    
    for i in range(0,100000):
        try:
            bsObj = BeautifulSoup(driver2.page_source,'html.parser')
            obj2 = bsObj.findAll('li', {'id' : re.compile('sp_nws.*')})
            for j in obj2:
                result.append(j.find('a').attrs['href'])
            driver2.find_element_by_class_name('next').click()
        except:
            break
    return(result)

In [258]:
def Text_analytics(url):  ## 2번째 Function , url을 받아와서
    article = Article(url)
    try:
        article.download()
        article.parse() ## url에 있는 기사 가져옴
        if(len(article.text) != 0):
            lexrank = LexRank()  # can init with various settings
            try:
                lexrank.summarize(article.text) ## 요약
                summaries = lexrank.probe(0)
                trans = []
                for summary in summaries: ## 요약된 문장마다 구글번역기로 영어로 번역
                    trans.append(translator.translate(summary, dest='en').text)
                L_to_S = "".join(trans) ## 문장이 list로 각각 들어가 있는데, 한 String으로 만들려고 하는 짓.
            except:
                L_to_S = translator.translate(article.text, dest='en').text

            a1 = driver.find_element_by_class_name('user-input-text-api') ## 감정분석하는 이 네모안을 찾음
            a1.clear() ## 기존에 있는 Text 지우기
            a1.send_keys(L_to_S) ## 그리고 위에서 한 String으로 만든 것들을 채워 넣음
            driver.find_element_by_xpath('//*[@id="main"]/section[1]/div[3]/div[1]/form/div[2]/input').click() ## '분석'을 클릭
            time.sleep(3) ## 분석 한 뒤에 약간 시간이 있어야 결과가 나와서 5초간 기다림
            percent = driver.find_element_by_class_name('absolutely-centered') ## 기다린 후에 %를 가져옴
            return(percent.text[:-2])
        else:
            return('-1')
    except:
        return('-1')

In [218]:
url = 'http://biz.heraldcorp.com/view.php?ud=20150217000753&ACE_SEARCH=1'
article = Article(url)
article.download()
article.parse()
article.text

"게임빌(대표 송병준, www.gamevil.com )이 자사 간판 게임 '별이되어라!'의 출시 1주년 맞이 대규모 프로모션을 진행 중이라고 17일, 밝혔다.게임빌은 365일 꾸준히 보내온 유저들의 성원에 보답하기 위해 대규모 이벤트와 업데이트를 마련했다.앞서 10일부터 접속하는 유저들에게 일별로 '골드', '신발', '티켓', 'S~SS급 동료 뽑기권' 등 푸짐한 아이템을 선물하고 있으며, 특히 출시 1주년이자 이벤트 마지막 날인 17일에는 접속하는 모든 유저들에게 'SSS급 동료 뽑기권'과 '추가 버프'를 파격적으로 제공한다.더불어 스킬 카드 합성 시스템 추가, 마스터 업적 추가, 요일 던전 업그레이드, 업적 시스템 개선 등 유저 맞춤형 업데이트도 동시에 진행, 본격적인 세몰이에 나섰다.지난 해 2월에 출시되어 장기 흥행 중인 '별이되어라!'는 국내 모바일게임 시장에서 RPG 인기를 선도해 왔다.플린트(대표 김영모)가 개발하고 게임빌이 서비스하는 이 게임은 양사의 긴밀한 파트너십을 바탕으로 개발사와 퍼블리셔 협력의 모범 사례로도 꼽히고 있다. 특히 국내 기세에 이어 글로벌 시장 공략도 앞두고 있어 관심이 집중된다.윤아름 기자 game@heraldcorp.com"

In [235]:
article.publish_date.strftime("%Y/%m/%d")

'2015/02/17'

In [249]:
Text_analytics('http://biz.heraldcorp.com/view.php?ud=20150217000753&ACE_SEARCH=1')

['2015/02/17', '100']

In [248]:
Text_analytics('http://www.fnnews.com/news/201502171433050639')

['2015/02/19', '-1']

In [ ]:
sum(summaries, [])

In [ ]:
trans = []
for summary in summaries:
    trans.append(translator.translate(summary, dest='en').text)
L_to_S = "".join(trans)
L_to_S

In [ ]:
start_time = time.time()
sentiment = []

ready = Find_URL('네이버',126) # 1번째 함수
data = pd.DataFrame(ready)

print("URL 수집 완료")
print("총 " + str(data.shape[0]) + " 번 돌아갑니다.")
i = 0
for url in data[2]:
    i += 1
    if(i % 10 == 0):
        print(str(i) + "번 째 돌아가고 있습니다.")
    try:
        p = Text_analytics(url) # 2번째 함수
        sentiment.append(p)
        print(p)
    except:
        sentiment.append('None')
        pass

data['sentiment'] = sentiment
end_time = time.time()
print(end_time - start_time)

In [ ]:
sentiment_1 = sentiment

In [ ]:
start_time = time.time()
sentiment = []

i = 0
for url in data[2]:
    i += 1
    if(i % 10 == 0):
        print(str(i) + "번 째 돌아가고 있습니다.")
    try:
        sentiment.append(Text_analytics(url))
    except:
        sentiment.append('None')
        pass

sentiment_2 = sentiment
end_time = time.time()
print(end_time - start_time)

In [ ]:
sentiment_2 = sentiment

In [ ]:
allsenti = sentiment_1 + sentiment_2

In [ ]:
len(allsenti)

In [ ]:
data['sentiment'] = allsenti

In [ ]:
data.columns = ['Title','Date','URL','Sentiment']

In [ ]:
data.to_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Naver_Sentiment.csv', index = False)

In [ ]:
start_time = time.time()
sentiment = []

ready = Find_URL('네이버',126)
data = pd.DataFrame(ready)

print("총 " + str(data.shape[0]) + " 번 돌아갑니다.")
i = 0
for url in data[2]:
    i += 1
    if(i % 10 == 0):
        print(str(i) + "번 째 돌아가고 있습니다.")
    try:
        sentiment.append(Text_analytics(url))
    except:
        sentiment.append('None')
        pass

data['sentiment'] = sentiment
end_time = time.time()
print(end_time - start_time)

In [ ]:
mydata = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Naver_Sentiment.csv')

In [ ]:
mydata

In [ ]:
news = 'https://search.naver.com/search.naver?ie=utf8&where=news&query=%ED%95%B8%EB%94%94%EC%86%8C%ED%94%84%ED%8A%B8&sm=tab_pge&sort=1&photo=0&field=1&reporter_article=&pd=5&ds=2016.11.15&de=2017.11.15&docid=&nso=so:dd,p:1y,a:t&mynews=1&start=71&refresh_start=0'
req = Request(news)
response = urlopen(req)
thepage = response.read()
bsObj = BeautifulSoup(thepage,'html.parser')

In [ ]:
obj2 = bsObj.findAll('li', {'id' : re.compile('sp_nws.*')})
len(obj2)

In [ ]:
for i in obj2:
    print(i.find('a').attrs['href'])

In [ ]:
obj2[0].find('a').attrs['href']

In [ ]:
obj2[0].find('a', {'class' : '_sp_each_url'}).text

In [ ]:
obj2[0].find('a', {'class' : '_sp_each_url'})

In [ ]:
round(int(bsObj.find('div', {'class' : 'title_desc all_my'}).text.split('/')[1][:-1]) / 10)

In [ ]:
bsObj.find('div', {'class' : 'title_desc all_my'}).text.split('/')[1][:-1]

In [ ]:
article = Article('http://www.newspim.com/news/view/20170904000197')
article.download()
article.parse() ## url에 있는 기사 가져옴

In [ ]:
article.publish_date.strftime("%Y/%m/%d")

In [ ]:
article.title

In [149]:
companyList = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/data/companyList.csv')

In [150]:
companyList

,compCode,compCredit,compName,creditDate
0,78340,AA0,(주)컴투스,2017.03.16
1,78340,A0,(주)컴투스,2016.03.17
2,78340,A0,(주)컴투스,2015.03.24
3,78340,AA0,(주)컴투스,2014.05.01
4,92130,AA-,(주)이크레더블,2017.03.03
5,92130,AA-,(주)이크레더블,2016.03.03
6,92130,AAA+,(주)이크레더블,2015.03.12
7,92130,A0,(주)이크레더블,2014.05.08
8,69080,A-,(주)웹젠,2017.04.01
9,69080,BBB+,(주)웹젠,2016.03.16


In [148]:
companyList['compName']
compName2 = []
for i in companyList['compName']:
    compName2.append(i.replace("(주)",""))

In [6]:
compName = companyList['compName'].unique()

In [15]:
compName2 = []
for i in compName:
    compName2.append(i.replace("(주)",""))

In [197]:
for i in range(7,8):
    compURL = []
    compName = companyList[i:(i+1)]['compName'].item().replace("(주)","")
    creditDate = companyList[i:(i+1)]['creditDate'].item()
    if(compName == "카카오"):
        continue
    try:
        compURL = Find_URL2(compName,creditDate)
    except:
        continue
    pd.DataFrame(compURL).to_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/' + compName + creditDate + '.csv')

In [46]:
companyList.shape[0]

156

In [193]:
companyList['compName'][6]

'(주)이크레더블'

In [52]:
result = []
for i in range(0,100000):
        try:
            bsObj = BeautifulSoup(driver2.page_source,'html.parser')
            obj2 = bsObj.findAll('li', {'id' : re.compile('sp_nws.*')})
            for j in obj2:
                result.append(j.find('a').attrs['href'])
            driver2.find_element_by_class_name('next').click()
        except:
            break

In [54]:
pd.DataFrame(result).to_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/data/' + '이크레더블' + '전체' + '.csv')

In [139]:
import os
allList = os.listdir('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/')

In [164]:
allList2 = allList[1:]

In [165]:
len(allList2)

150

In [201]:
a = 0
for i in allList2:
    length = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/' + i).shape[0]
    if(length > 100):
        a += 100
    else:
        a += length

In [245]:
allList2[60][:-14]

'아프리카티비'

In [229]:
random.sample(range(0,10),5)

[0, 2, 8, 9, 6]

In [203]:
test = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/게임빌2017.03.29.csv')

In [ ]:
for i in allList2[4:20]:
    allResult = []
    print(i[:-14] + ' 기업이 돌고 있습니다.')
    allURL = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/' + i)['0']
    if(len(allURL) > 100):
        allURL2 = allURL[random.sample(range(0,len(allURL)),100)]
    else:
        allURL2 = allURL
    for j in allURL2:
        allResult.append([i[:-14],Text_analytics(j)])
    pd.DataFrame(allResult).to_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment/' + i)

SK텔레콤 기업이 돌고 있습니다.
SK텔레콤 기업이 돌고 있습니다.
SK텔레콤 기업이 돌고 있습니다.
SK텔레콤 기업이 돌고 있습니다.
You must `download()` an article first!


In [65]:
import os
allSentiment = os.listdir('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment/')[1:]
allURL = os.listdir('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/')

In [76]:
import os
allSentiment2 = os.listdir('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment2/')
allURL = os.listdir('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/')

In [77]:
allSentiment2

['LG유플러스2015.06.10.csv',
 'LG유플러스2016.06.28.csv',
 'LG유플러스2017.01.09.csv',
 'SK텔레콤2014.10.15.csv',
 'SK텔레콤2015.10.26.csv',
 'SK텔레콤2016.05.20.csv',
 'SK텔레콤2017.04.12.csv',
 '이지웰페어2016.05.23.csv',
 '이지웰페어2017.01.20.csv',
 '이지웰페어2017.05.08.csv']

In [9]:
l = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment/' + allSentiment[0])['1']

In [67]:
s = []
for i in allSentiment:
    l = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment/' + i, encoding = "euc-kr")['1']
    allURL = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/' + i)['0']
    if(len(l[l != -1]) == 0):
        mean_Sentiment = -1
    else:
        mean_Sentiment = sum(l[l != -1]) / len(l[l != -1])
    URL_length = len(allURL)
    compName = i[0:-14]
    creditDate = i[-14:-4]
    s.append([compName, creditDate, URL_length, mean_Sentiment])

In [78]:
s2 = []
for i in allSentiment2:
    l = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment2/' + i, encoding = "utf-8")['1']
    allURL = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/URL/' + i)['0']
    if(len(l[l != -1]) == 0):
        mean_Sentiment = -1
    else:
        mean_Sentiment = sum(l[l != -1]) / len(l[l != -1])
    URL_length = len(allURL)
    compName = i[0:-14]
    creditDate = i[-14:-4]
    s2.append([compName, creditDate, URL_length, mean_Sentiment])

In [63]:
allSentiment[60]

'액토즈소프트2015.03.20.csv'

In [62]:
af = pd.read_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/Sentiment/' + allSentiment[60], encoding = "euc-kr")['1']

In [73]:
pd.DataFrame(s+s2).to_csv('/Users/hsw/Desktop/인하대학교/2017년 2학기/빅데이터 자료분석/Project/allSentiment_ver2.csv', index = False)

In [79]:
s2

[['LG유플러스', '2015.06.10', 1018, 76.35443037974683],
 ['LG유플러스', '2016.06.28', 1147, 85.060240963855421],
 ['LG유플러스', '2017.01.09', 866, 86.285714285714292],
 ['SK텔레콤', '2014.10.15', 1471, 82.402439024390247],
 ['SK텔레콤', '2015.10.26', 1471, 84.975903614457835],
 ['SK텔레콤', '2016.05.20', 1424, 85.759036144578317],
 ['SK텔레콤', '2017.04.12', 1043, 89.265060240963862],
 ['이지웰페어', '2016.05.23', 103, 92.565656565656568],
 ['이지웰페어', '2017.01.20', 91, 94.045454545454547],
 ['이지웰페어', '2017.05.08', 45, 83.909090909090907]]

In [138]:
s3 = s+s2

In [154]:
z = pd.DataFrame(s3)

In [168]:
a = list(z[0])

In [144]:
pd.DataFrame(s3)

,0,1,2,3
0,게임빌,2014.04.29,665,83.855670
1,게임빌,2015.06.22,659,84.161616
2,게임빌,2016.03.30,470,91.402062
3,게임빌,2017.03.29,232,88.040000
4,나이스정보통신,2014.05.30,16,96.466667
5,나이스정보통신,2015.03.31,29,87.440000
6,나이스정보통신,2016.03.18,6,71.166667
7,나이스정보통신,2017.03.18,6,71.166667
8,네오디안테크놀로지,2016.04.24,12,97.000000
9,네오디안테크놀로지,2017.04.20,8,82.375000
